In [1]:
import math

import numpy as np
import scipy as sp
import scipy.optimize as spo
import sympy as sym
from IPython.display import display, Latex

In [2]:
# p1x, p1y, p2x, p2y, d = sym.symbols("p1x p1y p2x p2y d", real=True)
# x1, y1 = sym.symbols("x1 y1", real=True)
# a, b, c = sym.symbols("a b c", real=True)

In [3]:
class Point:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y
    

In [4]:
class Line:
    def __init__(self, a: float, b: float, c: float):
        # a * x + b * y + c = 0
        self.a = a
        self.b = b
        self.c = c
    

In [5]:
class LineBuilder:
    @staticmethod
    def from_two_points(p1: Point, p2: Point) -> Line:
        a = p1.y - p2.y
        b = p2.x - p1.x
        c = p1.x * p2.y - p1.y * p2.x
        return Line(a, b, c)

In [9]:
class PerpendicularDistanceConstraint:
    def __init__(self, l: Line, d: float):
        self.l = l
        self.d = d
    
    def error(self, x: np.array):
        x0 = x[0]
        y0 = x[1]

        return abs(abs(self.l.a * x0 + self.l.b * y0 + self.l.c) / np.sqrt(self.l.a ** 2 + self.l.b ** 2) - self.d)
    
    def jacobian(self, x: np.array):
        x0 = x[0]
        y0 = x[1]
        s = np.sqrt(self.l.a ** 2 + self.l.b ** 2)
        t = self.l.a*x0 + self.l.b*y0 + self.l.c
        return np.array([math.copysign(self.l.a, t)/s, math.copysign(self.l.b, t)/s])

In [10]:
class PositiveQuadrantConstraint:
    def __init__(self):
        pass
    
    def _positive_error(self, x: float):
        if x > 0:
            return 0.
        return abs(x)
    
    def error(self, x: np.array):
        return self._positive_error(x[0]) + self._positive_error(x[1])
    
    def jacobian(self, x: np.array):
        return np.array(np.sign(x[0]), np.sign(x[1]))

In [11]:
def new_section():
    l1 = LineBuilder.from_two_points(Point(0,0),Point(2,1))
    l2 = LineBuilder.from_two_points(Point(0,0),Point(1,2))
    pdc1 = PerpendicularDistanceConstraint(l1, 1)
    pdc2 = PerpendicularDistanceConstraint(l2, 1)
    pqc1 = PositiveQuadrantConstraint()
    
    def error(x: np.array):
        err = pdc1.error(x) + pdc2.error(x) + pqc1.error(x)
        print(f"err at {x} is {err}")
        return err
        
        
    def jacobian(x: np.array):
        jac = pdc1.jacobian(x) + pdc2.jacobian(x) + pqc1.jacobian(x)
        print(f"jac at {x} is {jac}")
        return jac
    
    
    start = np.array([2.,2.], dtype=float)
    
    print(error(start))
    print(jacobian(start))
    
    result = spo.minimize(error, start, jac=jacobian, method="BFGS", options={
        "disp": True,
        "return_all": True
    })
    
    return(result)
new_section()

err at [2. 2.] is 0.2111456180001683
0.2111456180001683
jac at [2. 2.] is [0.5527864 1.4472136]
[0.5527864 1.4472136]
err at [2. 2.] is 0.2111456180001683
jac at [2. 2.] is [0.5527864 1.4472136]
err at [1.63960969 1.05648592] is 0.7942693898816903
jac at [1.63960969 1.05648592] is [0.5527864 1.4472136]
err at [1.94903522 1.86657248] is 0.29360836236183707
jac at [1.94903522 1.86657248] is [0.5527864 1.4472136]
err at [1.99279279 1.98113128] is 0.2228071288758382
jac at [1.99279279 1.98113128] is [0.5527864 1.4472136]
err at [1.99898079 1.99733167] is 0.2127947364833821
jac at [1.99898079 1.99733167] is [0.5527864 1.4472136]
err at [1.99985587 1.99962266] is 0.21137882892803805
jac at [1.99985587 1.99962266] is [0.5527864 1.4472136]
err at [1.99997962 1.99994664] is 0.21117859764197788
jac at [1.99997962 1.99994664] is [0.5527864 1.4472136]
err at [1.99999712 1.99999245] is 0.21115028183296458
jac at [1.99999712 1.99999245] is [0.5527864 1.4472136]
err at [1.99999959 1.99999893] is 0.21

  allvecs: [array([2., 2.])]
      fun: 0.2111456180001683
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([0.5527864, 1.4472136])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 55
      nit: 0
     njev: 43
   status: 2
  success: False
        x: array([2., 2.])